In [21]:
from __future__ import division
import numpy as np
import scipy.io as sio
from scipy import stats
#import xgboost as xgb
from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq
import pandas as pd
import math
import pickle
from sklearn.metrics import roc_auc_score
import copy
#from libsvm.svmutil import *
import csv
import sys
import re
import scipy
path='/Users/nhatcuong/Downloads/python_codes 2/data/'

In [2]:

mat = scipy.io.loadmat('/Users/nhatcuong/Downloads/python_codes 2/data/classification_data_sets_new.mat')
def rc(dna):
    complements = str.maketrans('ACGT', 'TGCA')
    rcseq = dna.translate(complements)[::-1]
    return rcseq
    
def read_data(F_GENOME):
    """
    Write your solution for Task A in this method.
    
    Parameters:
        - DO NOT modify the input parameters to this function
        - F_GENOME: A path to the file containing the FASTA-formatted genome
    
    Returns:
        - A single dimension list containing discovered off-targets
    
    """

    _TASK_A_CACHE=-1
    if _TASK_A_CACHE > 0:
        return _TASK_A_CACHE
    else:
        results = []
        #candidateGuides=[]
    
        # Add your code here
        # Opening the FASTA file and reading in the sequence
        seq = ''
        with open(F_GENOME,'r') as fp:
            for line in fp:
                if line[0] != '>':
                    seq += line.strip()
                    
        #print(f"Input sequence length: {len(seq)}")
    
        opposite = rc(seq)

        for pattern, strand in [
        [r"(?=([ATCG]{21}GG))", '+'], 
        [r"(?=(CC[ACGT]{21}))", '-']]:
            p = re.compile(pattern)
            for m in p.finditer(seq):
                if strand == '+':
                    results.append(
                        seq[m.start() : m.start() + 20] #m.group()[:-3]
                    )

                if strand == '-':
                    results.append(
                        rc(
                            seq[m.start() : m.start() + 20] #m.group()[:-3]
                        ))
        
        # I tried to extract guides with occurence > 1, implying that it is certain
        # this will return a list of off targets, however, it is too expensive
        # can=[]
        # for guide in results:
        #     if results.count(guide) > 1:
        #         can.append(guide)
        # results = results 
        #Cache the results to reduce run-time later

        return results


In [9]:
def countGC(input_gene, start):
    # s = open(input_gene).read()
    s=input_gene
    end = len(s)
    # 'compute the GC counts and GC contents of a sequences from (start,end)'
    GCcounts = len(s[start:end].replace('A', '').replace('T', ''))
    GCcontent = GCcounts / len(s)
    return GCcounts, GCcontent
def make_seq(input_genome):
    seq=''
    with open(input_genome,'r') as fp:
        for line in fp:
            if line[0] != '>':
                seq += line.strip()
    return seq
def ncf(s, order):
    # nucleotide composition features and position nucletide binary features
    # s--sequence
    # order--1,2,3: A,AA,AAA
    t_list = ["A", "G", "C", "T"]
    L = len(s)
    if order == 1:
        nc = t_list
    elif order == 2:
        nc = [m + n for m in ["A", "G", "C", "T"] for n in ["A", "G", "C", "T"]]
    elif order == 3:
        nc = [m + n + k for m in ["A", "G", "C", "T"] for n in ["A", "G", "C", "T"] for k in ["A", "G", "C", "T"]]
    nc_f = np.zeros((1, 4 ** order))

    pos_fea = np.zeros((1, 1))
    for i in range(0, L - order + 1):
        pos = np.zeros((1, 4 ** order))
        for j in range(0, len(nc)):
            if s[i:i + order] == nc[j]:
                nc_f[0][j] = nc_f[0][j] + 1
                pos[0][j] = 1
                pos_fea = np.hstack((pos_fea, pos))

    n = len(pos_fea[0])
    pos_fea = pos_fea[0][1:n]
    return nc_f, pos_fea

In [10]:
INPUT_GENOME = r"./S_bongori.fa" #My ending ID is 71
INPUT_GUIDES = r"./S_bongori.fa-27-guides.txt" #The fifth and sixth digits of my ID are 87 ~> 27
result = read_data(INPUT_GENOME)
sequence = make_seq(INPUT_GENOME)
# count_full,content_full = countGC()

In [11]:
nc_f = []
pos_fea=[]
for seq in result[:100]:
    nc,pos = ncf(seq,2)
    nc_f.append(nc)
    pos_fea.append(pos)


In [12]:
count=[]
content=[]
for seq in result[:100]:
    GCcount,GCcontent = countGC(seq,0)
    count.append(GCcount)
    content.append(GCcontent)


In [14]:
def binary_seqs(obs_seqs, t):
    nts = ['A', 'C', 'G', 'T']
    N = len(obs_seqs)
    n = len(obs_seqs[0])
    nc = []
    if t == 1:
        nc = nts
    elif t == 2:
        nc = [m + l for m in nts for l in nts]

    alpStat = np.zeros((N, 1 + n - t))
    for i in range(0, N):
        seq = obs_seqs[i]
        for j in range(0, 1 + n - t):
            mer = seq[j:j + t]
            ind = nc.index(mer)
            alpStat[i, j] = ind

    return alpStat
alpStat=[]
for seq in result[:100]:
    a = binary_seqs(seq,2)
    alpStat.append(a)

In [30]:
def get_sequences(fname):
    df = pd.read_excel(fname)
    targets = df['seq'].apply(lambda x: x.split(",")[0][:20]).unique()
    # targets = map(lambda x: ("NONE", x.upper(), 'AGG'), targets)
    return targets

In [24]:
fname=path+'guide_text.xlsx'


In [31]:
target = get_sequences(fname)

In [32]:
target 

array(['GAACACAAAGCATAGACTGC', 'GACCCCCTCCACCCCGCCTC',
       'GAGTCCGAGCAGAAGAAGAA', 'GGAATCCCTTCTGCAGCACC',
       'GGCACTGCGGCTGGAGGTGG', 'GGCCCAGACTGAGCACGTGA',
       'GGGAAAGACCCAGCATCCGT', 'GGGTGGGGGGAGTTTGCTCC',
       'GGTGAGTGAGTGTGTGCGTG'], dtype=object)

In [37]:
v1 =pd.read_excel(path+"V1_data.xlsx")

/Users/nhatcuong/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [38]:
v1

,Sequence,Target,30mer,Strand,Transcript,CutSite,Annotation,MOLM13 CD15,MOLM13 CD33,NB4 CD33,TF1 CD33,NB4 CD13,TF1 CD13
0,ACGTACAGGCCCCTGTCATT,CD13,AAAAACGTACAGGCCCCTGTCATTGGGGGTGAGG,antisense,ENST00000300060,8594,CDS,NaN,NaN,NaN,NaN,0.367491,-1.014438
1,CGTACAGGCCCCTGTCATTG,CD13,AAAACGTACAGGCCCCTGTCATTGGGGGTGAGGT,antisense,ENST00000300060,8593,CDS,NaN,NaN,NaN,NaN,1.291343,0.785962
2,GTACAGGCCCCTGTCATTGG,CD13,AAACGTACAGGCCCCTGTCATTGGGGGTGAGGTA,antisense,ENST00000300060,8592,CDS,NaN,NaN,NaN,NaN,1.493360,2.035237
3,CGCTGAAACCCGATTCCTAC,CD13,AACACGCTGAAACCCGATTCCTACCGGGTGACGC,sense,ENST00000300060,8558,CDS,NaN,NaN,NaN,NaN,-0.789269,-0.997515
4,TCAATGTGACGGGCTATTAC,CD13,AACCTCAATGTGACGGGCTATTACCGGGTGAACT,sense,ENST00000300060,15398,CDS,NaN,NaN,NaN,NaN,0.683439,-0.418296
...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,CCTCACTAGACTTGACCCAC,CD33,TTCTCCTCACTAGACTTGACCCACAGGCCCAAAA,sense,ENST00000262262,739,CDS,NaN,2.681098,2.597234,2.843958,NaN,NaN
878,TGTCGTAGTAGGGTATGGGA,CD33,TTCTTGTCGTAGTAGGGTATGGGATGGAAGAAAG,antisense,ENST00000262262,246,CDS,NaN,2.970595,2.813329,3.004306,NaN,NaN
879,TCTAGCTTGTTTGTGGCCAC,CD33,TTGATCTAGCTTGTTTGTGGCCACTGGAGAGTCC,antisense,ENST00000262262,328,CDS,NaN,2.329461,2.245616,2.635855,NaN,NaN
880,TGCCGTGGTCCTGGGGCCGT,CD33,TTGGTGCCGTGGTCCTGGGGCCGTGGGGTGATTA,antisense,ENST00000262262,918,CDS,NaN,1.808257,2.370927,2.828231,NaN,NaN


In [50]:
al=binary_seqs(result[1],1)

In [68]:
def TranEmis(alpStat, t, au=5, ad=5):
    N = len(alpStat[:, 0])
    n = len(alpStat[0, :])
    b = np.zeros((1, n + 2))
    c = np.zeros((n + 1, 1))
    d = np.eye(n + 1)
    Tr = np.hstack((c, d))
    Tr = np.vstack((Tr, b))
    tr = Tr.copy()
    tr[tr == 0] = float('-inf')
    tr[tr == 1] = 0

    Et = np.zeros((n + 2, 4 ** t))
    et = Et.copy()
    et[et == 0] = float('-inf')
    for i in range(1, n + 1):
        for j in range(0, 4 ** t):
            count = np.where(alpStat[:, i - 1] == j)
            p = (len(count[0]) + au) / (N + ad)
            if p == 0:
                et[i, j] = -0.01
            else:
                et[i, j] = math.log(p)

    return tr, et

In [69]:
TranEmis(al,1,au=5,ad=4)

(array([[-inf,   0., -inf],
        [-inf, -inf,   0.],
        [-inf, -inf, -inf]]),
 array([[       -inf,        -inf,        -inf,        -inf],
        [-0.98082925, -0.69314718, -0.98082925, -0.87546874],
        [       -inf,        -inf,        -inf,        -inf]]))